In [1]:
import sympy as sp
from IPython.display import display, Math
from generatePoly import generatePoly

In [2]:
w, s, e = sp.symbols('w s e') # Creamos las variables que usaremos en nuestras funciones.
H, f, Dem, p, np, result = sp.symbols('H, f, Dem p np result', cls=sp.Function) # Creamos ls funciones que usaremos.

Dada la función:

$$\left|H(s)\right|^2 = \frac{1}{14s^4+4s^2+2}$$

Lo podríamos separar en:

$$H(s)\cdot H(− s) = \frac{1}{14s^4+4s^2+2} = \frac{1}{a_1s^2 + a_0s + a} \cdot \frac{1}{a_1s^2 - a_0s + a}$$

Esto lo podemos generalizar para cualquier polinomio de grado n par. Primero ponemos el polinomio de $|H(s)|^2$ en una variable de sympy de tipo poly

In [143]:
Dem = sp.poly(4*s**4 + 4*s**2 + 2, s)
display(Dem)

Poly(4*s**4 + 4*s**2 + 2, s, domain='ZZ')

Luego generamos los polinomios genéricos pra $H(s)$ y $H(-s)$

In [7]:
p2, symbols = generatePoly(2)
np2 = sp.poly(p2.subs({s:-s}))

# Mostramos en LaTeX los polinomios:
display(Math((sp.latex(p2)))) 
display(Math((sp.latex(np2))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Multiplicamos ambos polinomios y guardamos en un nueva variable:

In [89]:
result = sp.poly(p2 * np2, s)
display(Math((sp.latex(result))))

<IPython.core.display.Math object>

Con eso ya podemos hacer nuestro sistema de ecuaciones que será algo así:

In [104]:
def printCoefArray (poly, dem):
    symbolsFormatted = {symbol: "{" + sp.latex(symbol) + "}" for symbol in symbols}
    result = []
    n = sp.degree(dem)
    for i in range(n+1):
        polyCoeff = poly.coeff_monomial(s**i)
        demCoeff = dem.coeff_monomial(s**i)
        equation = polyCoeff - demCoeff
        if (equation != 0):
            equation = sp.latex(demCoeff) + "=" + sp.latex(polyCoeff, symbol_names=symbolsFormatted)
            display(Math(equation))
            
printCoefArray(result,Dem)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Para luego poder usar __nonlinsolve__ vamos a necesitar que este array de coeficientes esté igualado a 0, en las siguiente linea nos encargamos de igualarlo a 0 y mostrarlo.

In [107]:
def coefArray (poly, dem):
    result = []
    n = sp.degree(dem)
    for i in range(n+1):
        polyCoeff = poly.coeff_monomial(s**i)
        demCoeff = dem.coeff_monomial(s**i)
        equation = polyCoeff - demCoeff
        if( equation  != 0):
            result.append(equation)
    return result

coefarray = coefArray(result, Dem) # Calculo los coeficientes

## Todo lo demás es sólo con fines de mostrar las ecuaciones de una forma cómoda y linda, no de importancia porque uso estructuras complejas.
symbolsFormatted = {symbol: "{" + sp.latex(symbol) + "}" for symbol in symbols}
equations = [f"Equation  \hspace{{0.2cm}} {i}:" + sp.latex(equation, symbol_names=symbolsFormatted) for i, equation in enumerate(coefarray) if equation is not None]
Math("\\\\\n".join(equations))


<IPython.core.display.Math object>

Solucionamos el sistema con nonlinsolve y listamos sólo aquellas donde todos los coeficientes sean positivos:

In [229]:
soluciones = sp.nonlinsolve(coefarray, symbols)
## No den importancia a lo que sigue, es sólo para limpiar las soluciones que no nos importan (complejas, negativas, etc)
def hasProperty(sol):
    return all((s.is_real and sp.GreaterThan(s, 0) for s in sol))
finalResult = list(filter(hasProperty, soluciones))[0]

#Imprimir la solución

for i,result in enumerate(finalResult):
    display(Math( f"a_{i} =" + sp.latex(result)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Reemplazamos en nuestro polinomio de $H(s)$ positivo:

In [129]:
finalResult_prettier = []

# Todo este for se encarga de que en vez de aparecer raiz de 2 y etc, salga el valor númerico gracias a sp.N, 
# sin embargo si lo aplicaba a los 3 números, saldría 2.0 en vez de 2, así que excluí los integers para que no 
# Ocurra en ellos.
for result in finalResult: 
    if type(result) != sp.core.numbers.Integer:
        result = sp.N(result,3)
        
    finalResult_prettier.append(result)
        
values = {symbol: finalResult_prettier[i] for i, symbol in enumerate(symbols)}

H = p2.subs(values)
H = 1/H
Mostrar = "H(S) ="+ sp.latex(H)
display(Math(Mostrar))

<IPython.core.display.Math object>